In [ ]:
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt
import sys
import os
import cv2
from skimage import data, filters
from skimage.segmentation import flood, flood_fill
from PIL import Image as im 

In [ ]:
# define constants and input, output paths
inpath = '/home/eikthedragonslayer/Desktop/CVS_data/CVS/Annotated/2'
outpath = '/home/eikthedragonslayer/Desktop/CVS_data/CVS/mask/2'
if not os.path.isdir(outpath):
    os.mkdir(outpath)

In [ ]:
def consecutive(data, stepsize=8):
    return np.split(data, np.where(np.diff(data) >= stepsize)[0]+1)

In [ ]:
# for the IVUS dataset
for img in os.listdir(inpath):
    print(img)
    basename = img.split('.')[0]
    if '_LI' in basename:
        basename = basename[:-3]
    image = plt.imread(os.path.join(inpath, img))
    image = image[0:499,0:499] # resize
    plt.imshow(image)
    plt.show()
    mask = image[:,:,0]-image[:,:,1]
    mask[mask > 230] = 0 # to reduce red inconsistence noise and find the pure red circle
    mask[mask < 50] = 0
    result = mask.copy()
    # create binary mask
    seed = (0,0)
    count = 0
    for i in range(result.shape[0]):
        if np.sum(result[i]) > 0:
            count += 1
            if count > 30:
                row = np.nonzero(result[i])[0]
                if row.shape[0] > 1:
                    groups = consecutive(row)
                    if len(groups) > 1:
                        seed = (i,groups[-1][0]-10)
                        break
    result = flood_fill(result, seed, 255)
    result = result-mask
    result[result > 0] = 255
    new_mask = np.zeros_like(image)
    new_mask[:,:,0] = result
    new_mask[:,:,1] = result
    new_mask[:,:,2] = result
    #plt.imshow(result)
    plt.imshow(new_mask)
    plt.show()
    print(np.sum(result > 0))
    print(np.sum(result == 255))
    bfn =basename+'.npy'
    fn = basename+'.jpeg'
    #print(fn)
    #sys.exit()
    out1 = os.path.join(outpath, fn)
    out2 = os.path.join(outpath, bfn)
    data = im.fromarray(new_mask)
    data.save(out1)
    result = result / 255
    with open(out2,'wb') as f:
        np.save(f,result)